# Algoritmo para la localización de un nuevo Restaurante


El objetivo de este proyecto es la elaboración de un flujo que permita obtener la información necesaria para, conocidos unos parámetros básicos, se pueda recomendar un local para el éxito de un nuevo restaurante en la ciudad de Madrid.

El proceso consistirá en:

1 - Recopilación de información

    1.1.- Información de locales disponibles (idealista)
    
    1.2.- Información de restaurantes (tripadvisor)

    1.3.- Información de restaurantes (el tenedor)
        
2 - Limpieza de datasets

3 - Unificación de datasets y reducción de features

4 - Clustering

In [2]:
# Librerías empleadas
import numpy as np
import pandas as pd

# 1 Obtención de los datos
## 1.1 - Idealista
Los datos de este portal se van a extraer a traves de la scrapear la web al hacer búsquedas por los distintos códigos postales de la ciudad. Para realizar las búsquedas partimos del enlace: https://www.idelista.com/buscar/ al que se le pueden añadir el tipo y el CP. Aunque en el código está implementeado la posibilidad de buscar locales tanto para alquiler como para compra, esta segunda opción no se ha ejecutado.

De cada página de búsqueda se obtendrán los enlaces para posteriormente scrapear la data deseada de los locales disponibles, así cómo el enlace para la siguiente página de la búsqueda ( se ha remarcao con un rectángulo de color la info acopiada):
<div>
<img src="idealista1-1.PNG" width="400"/>
</div>

De los enlaces de cada local se extrae la siguiente información:
<div>
<img src="idealista2-1.PNG" width="400"/>
    
<img src="idealista2-2.PNG" width="400"/>
</div>

El código que realiza el scraping de la página web, así como generar el dataset se encuentra [aquí](https://github.com/cmorenocobian/master/blob/545bcfa1dd668442234a990cd0bb550357d7ba4d/1.3%20idealista_scrapy_00.py)

In [3]:
# Los códigos postales de Madrid son
cp=''
with open('codigos_postales.txt', 'r') as f:
    for line in f.read():
        cp += line
cp=list(filter(None,cp.split('\n')))
print('Número de códigos postales: ',len(cp))
print('\n',cp)

Número de códigos postales:  55

 ['28001', '28002', '28003', '28004', '28005', '28006', '28007', '28008', '28009', '28010', '28011', '28012', '28013', '28014', '28015', '28016', '28017', '28018', '28019', '28020', '28021', '28022', '28023', '28024', '28025', '28026', '28027', '28028', '28029', '28030', '28031', '28032', '28033', '28034', '28035', '28036', '28037', '28038', '28039', '28040', '28041', '28042', '28043', '28044', '28045', '28046', '28047', '28048', '28049', '28050', '28051', '28052', '28053', '28054', '28055']


In [16]:
# El resultado sería:
idealista = pd.read_json('data_idealista.json')
idealista.head(5)

,id,url,postcode,title,address,price,floor_area,location,key_features,coordinates,date
0,95342255,https://www.idealista.com/inmueble/95342255/?x...,28001,Alquiler de Local en calle de Goya,"Recoletos, Madrid",6.500,300,"[Calle de Goya, Barrio Recoletos, Distrito Bar...","[300 m2 construidos, Segunda mano/buen estado,...","{'latitude': 40.424265, 'longitude': -3.6857166}",2021-09-21
1,95340968,https://www.idealista.com/inmueble/95340968/?x...,28001,Alquiler de Local en calle del Conde de Aranda...,"Recoletos, Madrid",400.000,15,"[Calle del Conde de Aranda, 22, Barrio Recolet...","[15 m2 construidos, 13 m2 utiles, 1 planta, Se...","{'latitude': '', 'longitude': ''}",2021-09-21
2,95327756,https://www.idealista.com/inmueble/95327756/?x...,28001,Alquiler de Local en Goya,"Barrio de Salamanca, Madrid",16.000,100,"[Barrio Goya, Distrito Barrio de Salamanca]","[100 m2 construidos, Segunda mano/buen estado,...","{'latitude': 40.4242938, 'longitude': -3.6782076}",2021-09-21
3,92137316,https://www.idealista.com/inmueble/92137316/?x...,28001,"Alquiler de Local en calle de Villanueva, 11","Recoletos, Madrid",17.394,724,"[Calle de Villanueva, 11, Barrio Recoletos, Di...","[724 m2 construidos, 1 planta, Segunda mano/bu...","{'latitude': 40.4225419, 'longitude': -3.6863847}",2021-09-21
4,95053883,https://www.idealista.com/inmueble/95053883/?x...,28001,Alquiler de Local en calle del Conde de Aranda,"Recoletos, Madrid",1.450,25,"[Calle del Conde de Aranda, Barrio Recoletos, ...","[25 m2 construidos, Segunda mano/buen estado, ...","{'latitude': 40.4202436, 'longitude': -3.6859767}",2021-09-21


In [5]:
# Se ha generado un dataset con:
print('Numero de locales incluidos en el dataset: ' + str(idealista.shape[0]))
print('Campos generados para cado uno de los restaurantes: ' + str(idealista.shape[1]))

Numero de locales incluidos en el dataset: 737
Campos generados para cado uno de los restaurantes: 11


In [17]:
# Compruebo los nulls
idealista.isnull().sum()

id              0
url             0
postcode        0
title           0
address         0
price           0
floor_area      0
location        0
key_features    0
coordinates     0
date            0
dtype: int64

No se ha podido sacar la geolocalización de todos los locales, por lo que se va a recurrir al [nomenclator](https://www.madrid.org/nomecalles/DescargaBDTCorte.icm) de la Comunidad para obtener las coordenadas por su dirección física.

## 1.2 - Tripadvisor

El proceso consiste en extaer de este portal toda la información necesaria. El objetivo es recopilar los datos de los restaurantes de la web. Para ello se debe buscar los restaurantes de Madrid en la citada web, [enlace](https://www.tripadvisor.es/Restaurants-g187514-Madrid.html), de esta primera página sólo se obtienen los enlaces a las páginas de los restaurantes y de la parte inferior el enlace a la página siguiente:

De cada página de búsqueda se extrae la siguiente información (marcada con un rectángulo de color):
<div>
<img src="trip1-1.PNG" width="400"/>
</div>
De cada página se pueden extraer 30 enlaces a las páginas de los restaurantes, antes de poder pasar a la siguiente

Y de cada uno de los enlaces se extrae los siguientes datos de los restaurantes:
<div>
<img src="trip2-1.PNG" width="400"/>
<img src="trip2-2.PNG" width="400"/>
</div>


El código que realiza el scraping de la página web, así como generar el dataset se encuentra [aquí](https://github.com/cmorenocobian/master/blob/545bcfa1dd668442234a990cd0bb550357d7ba4d/1.1%20tripadvisor_scrapy_01.py)

In [7]:
# El resultado sería:
tripadvisor = pd.read_json('data_tripadvisor.json')
tripadvisor.head(5)

,nombre,url,rating,review_count,nombre_web,rating_web,ranking_web,rango_precios,calle_web,ratings_web,Precio_web,min_price,max_price
0,Restaurante Chino Central,https://www.tripadvisor.es/Restaurant_Review-g...,6.121 de 10.471 Restaurantes en Madrid,15 opiniones,Restaurante Chino Central,3,6.136,"Detalles\nCOMIDAS\nComidas, Cenas\nVer todos l...","Palencia, 4, 28020 Madrid, España","{'Comida': '35', 'Servicio': '40', 'Calidad/pr...",NaN,NaN,NaN
1,Alpunto Sol,https://www.tripadvisor.es/Restaurant_Review-g...,6.122 de 10.471 Restaurantes en Madrid,5 opiniones,Alpunto Sol,4,6.138,NaN,"Puerta Del Sol, Madrid, España",{},NaN,NaN,NaN
2,Restaurante Hileras Michel,https://www.tripadvisor.es/Restaurant_Review-g...,6.123 de 10.471 Restaurantes en Madrid,5 opiniones,Restaurante Hileras Michel,4,6.139,Detalles\nTIPOS DE COCINA\nEspañola\nVer todos...,"Calle Hileras 6, 28013 Madrid, España",{},Española,NaN,NaN
3,All India,https://www.tripadvisor.es/Restaurant_Review-g...,6.124 de 10.471 Restaurantes en Madrid,7 opiniones,All India,3,6.140,NaN,"Calle Francisco Villaespesa numero 6, 28017 Ma...",{},Española,NaN,NaN
4,+Que Pan,https://www.tripadvisor.es/Restaurant_Review-g...,6.125 de 10.471 Restaurantes en Madrid,13 opiniones,+Que Pan,3,6.141,Detalles\nTIPOS DE COCINA\nEspañola\nVer todos...,"Plaza de La Puerta de Moros 3, 28005 Madrid, E...","{'Comida': '45', 'Servicio': '45', 'Calidad/pr...","€, Española",NaN,NaN


In [8]:
# Se ha generado un dataset con:
print('Numero de restaurantes incluidos en el dataset: ' + str(tripadvisor.shape[0]))
print('Campos generados para cado uno de los restaurantes: ' + str(tripadvisor.shape[1]))

Numero de restaurantes incluidos en el dataset: 6780
Campos generados para cado uno de los restaurantes: 13


In [9]:
tripadvisor.isnull().sum()

nombre              0
url                 0
rating              0
review_count        0
nombre_web          0
rating_web          0
ranking_web         1
rango_precios     719
calle_web           1
ratings_web         1
Precio_web          3
min_price        4117
max_price        4273
dtype: int64

## 1.3 El tenedor
    

El proceso consiste en extaer de este portal toda la información necesaria. El objetivo es recopilar los datos de los restaurantes de la web. Para ello se debe buscar los restaurantes de Madrid en la citada web, [enlace](https://www.thefork.es/search/?cityId=328022), de esta primera página sólo se obtienen los enlaces a las páginas de los restaurantes y de la parte inferior el enlace a la página siguiente:

De cada página de búsqueda se extrae la siguiente información (marcada con un rectángulo de color):
<div>
<img src="tenedor1.PNG" width="400"/>
</div>
De cada página se pueden extraer 30 enlaces a las páginas de los restaurantes, antes de poder pasar a la siguiente

Y de cada uno de los enlaces se extrae los siguientes datos de los restaurantes:
<div>
<img src="tenedor2.PNG" width="400"/>
</div>


El código que realiza el scraping de la página web, así como generar el dataset se encuentra [aquí](https://github.com/cmorenocobian/master/blob/main/1.3%20tenedor_scrapy_01.py)

In [10]:
# El resultado sería:
tenedor = pd.read_json('data_tenedor.json')
tenedor.head(5)

,nombre,url,direccion,tags,precio,rating,reviews,oferta
0,La venganza de Malinche - Jardines,https://www.eltenedor.es/restaurante/la-vengan...,"Calle Jardines, 5 28013 Madrid",[Mexicano],Precio medio 25 €,"8,6",437 Opiniones,NaN
1,La Lonchería - Prosperidad,https://www.eltenedor.es/restaurante/la-lonche...,"Calle de Vinaroz, 1 (Mercado de Prosperidad, P...",[Mexicano],Precio medio 15 €,"7,9",45 Opiniones,NaN
2,Il Bambino de la Bambola,https://www.eltenedor.es/restaurante/il-bambin...,"C/ Martínez de la Riva, 36 28053 Madrid",[Italiano],Precio medio 26 €,"8,3",954 Opiniones,NaN
3,Happy Green,https://www.eltenedor.es/restaurante/happy-gre...,"Calle San Marcos, 28 28004 Madrid",[Mediterráneo],Precio medio 12 €,"7,4",138 Opiniones,-30% en carta *
4,Raza 7 – Hotel Senator Diana,https://www.eltenedor.es/restaurante/raza-7-ho...,"Calle Galeón, 27 28042 Madrid","[Asador, Insider, Selección Insider - Terrazas]",Precio medio 45 €,"8,7",154 Opiniones,NaN


In [11]:
# Se ha generado un dataset con:
print('Numero de restaurantes incluidos en el dataset: ' + str(tenedor.shape[0]))
print('Campos generados para cado uno de los restaurantes: ' + str(tenedor.shape[1]))

Numero de restaurantes incluidos en el dataset: 4359
Campos generados para cado uno de los restaurantes: 8


# 2 Limpieza y unificación
## 2.1 Tripadvisor

Para ello se utiliza el [script](https://github.com/cmorenocobian/master/blob/main/2.1%20limpiador_tripadvisor.py) que se encuentra en github. Se inicia cargando los datos generados anteriormente y devuelve un dataframe (que se almacena como un archivo pickle.
Es importante destacar la reducción de categorías implementadas sobre este dataset, ya que en el proceso se independizan todas las categorías de los restaurantes, generando columnas dummy, que llegan a sobrepasar la centena. En primer lugar se utiliza una agrupación de valores conocidos y posteriormente se busca utilizar el algoritmo SVD (Singular Value Descomposition) acompañado de una función de maximización, pero los resultados no son los deseados (simplifica demasiado el modelo), cuando la información más importante es la diversificación culinaria.
El proceso se encuentra en el siguiente script de [2.1 limpiador_tripadvisor](https://github.com/cmorenocobian/master/blob/main/2.1%20limpiador_tripadvisor.py)

In [12]:
# El dataset lipio quedaría:
tripadvisor = pd.read_pickle('datos_tripadvisor.pkl')
tripadvisor.head(5)

,nombre,url,rating,review_count,rating_web,min_price,max_price,calle_tipo,calle_nom,calle_num,...,tipo_sueca,tipo_ucraniana,tipo_street food,tipo_australiana,tipo_europa oriental,precio_medio,r_comida,r_servicio,r_calidad/precio,r_atmósfera
0,Restaurante Chino Central,https://www.tripadvisor.es/Restaurant_Review-g...,6121,15,3,0,0,[calle],palencia,4,...,0,0,0,0,0,0.0,35,40,35,0
1,Alpunto Sol,https://www.tripadvisor.es/Restaurant_Review-g...,6122,5,4,0,0,[calle],del sol,0,...,0,0,0,0,0,0.0,0,0,0,0
2,Restaurante Hileras Michel,https://www.tripadvisor.es/Restaurant_Review-g...,6123,5,4,0,0,[calle],hileras,6,...,0,0,0,0,0,0.0,0,0,0,0
3,All India,https://www.tripadvisor.es/Restaurant_Review-g...,6124,7,3,0,0,[calle],francisco villaespesa numero,6,...,0,0,0,0,0,0.0,0,0,0,0
4,+Que Pan,https://www.tripadvisor.es/Restaurant_Review-g...,6125,13,3,0,0,[plaza],de la de moros,3,...,0,0,0,0,0,0.0,45,45,45,0


In [14]:
tripadvisor.isnull().sum()

nombre              0
url                 0
rating              0
review_count        0
rating_web          0
                   ..
precio_medio        0
r_comida            0
r_servicio          0
r_calidad/precio    0
r_atmósfera         0
Length: 108, dtype: int64

# 2.2 El tenedor
En este caso el proceso de limpieza se ha centrado más en extraer las direcciones de los locales, ya es necesario para poder juntar los dos dataset correctamente. El script es el denominado [2.2 limpiador_tenedor.py](https://github.com/cmorenocobian/master/blob/main/2.2%20limpiador_tenedor.py)

In [13]:
# El dataset resultante es:
tenedor = pd.read_pickle('data_tenedor.pkl')
tenedor.head(5)

,nombre,url,rating,reviews,mexicano,italiano,mediterraneo,asador,insider,terrazas,...,caribeño,tradicional,valenciano,ingles,tipo,calle,numero,cp,medio,descuento
0,La venganza de Malinche - Jardines,https://www.eltenedor.es/restaurante/la-vengan...,8.6,437.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,calle,jardines,5.0,28013.0,25.0,0.0
1,La Lonchería - Prosperidad,https://www.eltenedor.es/restaurante/la-lonche...,7.9,45.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,calle,de vinaroz,1.0,28002.0,15.0,0.0
2,Il Bambino de la Bambola,https://www.eltenedor.es/restaurante/il-bambin...,8.3,954.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,calle,martinez de la riva,36.0,28053.0,26.0,0.0
3,Happy Green,https://www.eltenedor.es/restaurante/happy-gre...,7.4,138.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,calle,san marcos,28.0,28004.0,12.0,30.0
4,Raza 7 – Hotel Senator Diana,https://www.eltenedor.es/restaurante/raza-7-ho...,8.7,154.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,calle,galeon,27.0,28042.0,45.0,0.0


In [15]:
tenedor.isnull().sum()

nombre       0
url          0
rating       0
reviews      0
mexicano     0
            ..
calle        0
numero       0
cp           0
medio        0
descuento    1
Length: 92, dtype: int64

# 3 Unificación de datasets

En este proceso se va a proceder a juntar los dos datasets, para ello se a van a utilizar dos metodologías distintas:
1 - Utilizando la librería FuzzyWuzzy que permitirá ver la similitud de cadenas de texto para poder unificar los nombres de los locales, también se utilizará para conseguir las coordenadas de los locales de los que se desconocen a través del nomenclator de la Comunidad de Madrid, un dataset que contiene las coordenadas de todas las direcciones de la comunidad. Se exigitá una similitud mínima del 90%
2 - Mediante la librería GeoPandas podremos ver la distancia ente todos los locales y seleccionar los locales más próximos con mismo nombre si la ditancia entre ellos es menor a un valor predefinido, en este caso hemos supuesto 10 m.

In [19]:
# Ejemplo funcionamiento FuzzyWuzzy para ver la similitud de dos cadenas de texto
import fuzzywuzzy
from fuzzywuzzy import process

nombres_tripadvisor = list(map(lambda x: x.lower().strip(),  tripadvisor['nombre'].unique()))
nombres_tenedor = list(map(lambda x: x.lower().strip(),  tenedor['nombre'].unique()))
nombres_dict = {}
for element in nombres_tenedor:  
    matches = process.extract(element,
                            nombres_tripadvisor, 
                            limit=3, 
                            scorer=fuzzywuzzy.fuzz.token_set_ratio)
    nombres_dict[element] = matches
    parecidos = []
    no_parecidos = []
    for element in nombres_dict:
        if nombres_dict[element][0][1] > 90:
            parecidos.append([element, nombres_dict[element][0][0]])
        elif nombres_dict[element][0][1] <90:
            no_parecidos.append([element, nombres_dict[element][0][0]])

In [22]:
#
for i in range(5,10):
    print(parecidos[i][0], ' - ', parecidos[i][1])

jojoto  -  jojoto arepa bar
casa paulino de alonso cano  -  casa paulino
atlántico casa de comidas  -  atlántico casa de comidas
juanyta me mata  -  juanyta me mata
empatía. rincón vegano  -  empatía. rincón vegano


In [ ]:
# Función para definir la distancia entre los dos locales más próximos
import geopandas as gdp
import shapely
from scipy.spatial import cKDTree

# Creo una función que busca el punto más cercano y la distancia entre ellos
# Con esta información podré unificar los restaurantes de los dos datasets

def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

In [37]:
#El resultado es el dataset con el que se va a trabajar y que reúne toda la información necesaria
restaurantes = pd.read_pickle('restaurantes.pkl')
restaurantes.head(5)

,index,nombre,media,rating,rating_web,review_count,calle_tipo,calle_nom,calle_num,calle_cp,...,tipo_Asiatica,tipo_Sur_americana,tipo_Africana,tipo_Norte_Americana,Tipo Vía,Vía,geometry,NUME_NUME,x,y
3995,72,Ramon Freixa Madrid,300.0,81,4,841,Calle,Claudio Coello,67,28001,...,0.0,0.0,0.0,0.0,Calle,Claudio Coello,POINT (441789.9738999996 4475544.6876),67.0,441789.9738999996,4475544.6876
4677,3185,Sunday Brunch at Unico,272.5,3930,4,28,Calle,Claudio Coello,67,28001,...,0.0,0.0,0.0,0.0,Calle,Claudio Coello,POINT (441789.9738999996 4475544.6876),67.0,441789.9738999996,4475544.6876
4181,2116,Restaurante Haroma,262.5,2548,4,34,Calle,Diego de León,43,28006,...,0.0,0.0,0.0,0.0,Calle,Diego de León,POINT (442404.3002000004 4476251.293099999),43.0,442404.3002000004,4476251.293099999
1109,100,Coque,300.0,116,4,212,Calle,Marqués del Riscal,11,28010,...,0.0,0.0,0.0,0.0,Calle,Marqués del Riscal,POINT (441436.193 4475794.3553),11.0,441436.193,4475794.3553
2934,140,La Terraza del Casino de Madrid,164.0,161,4,579,Calle,Alcalá,15,28014,...,0.0,0.0,0.0,0.0,Calle,Alcalá,POINT (440590.0938999997 4474370.9034),15.0,440590.0938999997,4474370.9034
